In [33]:
import os

os.environ['KAGGLE_API_TOKEN'] = "YOURKEY"
!pip install --upgrade kaggle
!kaggle config set -n api_token -v $KAGGLE_API_TOKEN


- api_token is now set to: KGAT_3e848ca33582561ac728cfeb5afe8a3d


In [34]:
!kaggle datasets list -s "imagenet"


ref                                                       title                                                size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  --------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
wanghaohan/imagenetsketch                                 ImageNet-Sketch                               15219198396  2019-06-18 13:45:50.200000           4395         67  0.625            
vitaliykinakh/stable-imagenet1k                           Stable ImageNet-1K                            10519983320  2022-09-08 22:19:54.953000           3864         48  0.9375           
akash2sharma/tiny-imagenet                                Tiny ImageNet                                   497536564  2018-09-27 12:11:20.917000          21597        225  0.3125           
deeptrial/miniimagenet                                 

In [35]:

!kaggle datasets download -d prasunroy/natural-images
!unzip -q natural-images.zip -d natural_images


Dataset URL: https://www.kaggle.com/datasets/prasunroy/natural-images
License(s): CC-BY-NC-SA-4.0
natural-images.zip: Skipping, found more recently modified local copy (use --force to force download)
replace natural_images/natural_images/airplane/airplane_0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [36]:
# move everything from nested folder to main folder
!mv natural_images/data/natural_images/* natural_images/natural_images/

# remove the empty folders
!rm -r natural_images/data/


mv: cannot move 'natural_images/data/natural_images/airplane' to 'natural_images/natural_images/airplane': Directory not empty
mv: cannot move 'natural_images/data/natural_images/car' to 'natural_images/natural_images/car': Directory not empty
mv: cannot move 'natural_images/data/natural_images/cat' to 'natural_images/natural_images/cat': Directory not empty
mv: cannot move 'natural_images/data/natural_images/dog' to 'natural_images/natural_images/dog': Directory not empty
mv: cannot move 'natural_images/data/natural_images/flower' to 'natural_images/natural_images/flower': Directory not empty
mv: cannot move 'natural_images/data/natural_images/fruit' to 'natural_images/natural_images/fruit': Directory not empty
mv: cannot move 'natural_images/data/natural_images/motorbike' to 'natural_images/natural_images/motorbike': Directory not empty
mv: cannot move 'natural_images/data/natural_images/person' to 'natural_images/natural_images/person': Directory not empty


In [40]:
import os
from PIL import Image

# Paths
source_dir = "natural_images/natural_images/person"
target_dir = "natural_images/cut"
os.makedirs(target_dir, exist_ok=True)


In [41]:
# Loop through all images
for img_name in os.listdir(source_dir):
    img_path = os.path.join(source_dir, img_name)
    if os.path.isfile(img_path):
        try:
            img = Image.open(img_path)
            img_width, img_height = img.size

            # Cut image into 50x50 patches
            patch_id = 0
            for top in range(0, img_height, 50):
                for left in range(0, img_width, 50):
                    # Ensure we don't go out of bounds
                    right = min(left + 50, img_width)
                    bottom = min(top + 50, img_height)

                    patch = img.crop((left, top, right, bottom))

                    # Save the patch
                    patch_filename = f"{os.path.splitext(img_name)[0]}_patch{patch_id}.png"
                    patch.save(os.path.join(target_dir, patch_filename))
                    patch_id += 1
        except Exception as e:
            print(f"Skipping {img_name}, error: {e}")

In [42]:
folder = "natural_images/cut"  # change to the folder you want to check
all_files = os.listdir(folder)

# Only count files, skip any subfolders
num_images = sum(1 for f in all_files if os.path.isfile(os.path.join(folder, f)))
print(f"Number of images: {num_images}")

Number of images: 35463


In [43]:
import numpy as np
from PIL import Image
import colorsys

def add_color_noise_hsl(img, noise_level=0.05):
    """
    Adds color noise in HSL space.

    Args:
        img: PIL.Image
        noise_level: float, max noise added to H, S, L (H in [0,1], S,L in [0,1])

    Returns:
        PIL.Image with noisy colors
    """
    img = img.convert("RGB")
    img_array = np.array(img).astype(np.float32) / 255.0
    noisy_array = np.zeros_like(img_array)

    # Process pixel by pixel
    for i in range(img_array.shape[0]):
        for j in range(img_array.shape[1]):
            r, g, b = img_array[i, j]
            h, l, s = colorsys.rgb_to_hls(r, g, b)  # note: hls in Python
            # Add noise
            h = (h + np.random.uniform(-noise_level, noise_level)) % 1.0
            s = np.clip(s + np.random.uniform(-noise_level, noise_level), 0, 1)
            l = np.clip(l + np.random.uniform(-noise_level, noise_level), 0, 1)
            r_new, g_new, b_new = colorsys.hls_to_rgb(h, l, s)
            noisy_array[i, j] = [r_new, g_new, b_new]

    noisy_array = (noisy_array * 255).astype(np.uint8)
    return Image.fromarray(noisy_array)

In [44]:
import os
from PIL import Image

source_dir = "natural_images/cut"
target_dir = "natural_images/cut_corrupt/color_noise"
os.makedirs(target_dir, exist_ok=True)

all_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

for i, img_name in enumerate(all_files):
    img_path = os.path.join(source_dir, img_name)
    try:
        img = Image.open(img_path)
        noisy_img = add_color_noise_hsl(img, noise_level=0.05)  # use HSL function

        save_path = os.path.join(target_dir, img_name)
        noisy_img.save(save_path)

        if (i+1) % 1000 == 0:
            print(f"{i+1} images processed")
    except Exception as e:
        print(f"Skipping {img_name}, error: {e}")


1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
10000 images processed
11000 images processed
12000 images processed


KeyboardInterrupt: 

In [46]:
!pip install kornia


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.5 MB/s eta 0:00:00


In [47]:
import os
from PIL import Image
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

source_dir = "natural_images/cut"
target_dir = "natural_images/cut_corrupt/color_noise"
os.makedirs(target_dir, exist_ok=True)

# List all images in source folder
all_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

# Skip already processed images
already_done = set(os.listdir(target_dir))
files_to_process = [f for f in all_files if f not in already_done]

print(f"Images left to process: {len(files_to_process)}")

def add_color_noise_hsl_gpu(img_tensor, noise_level=0.05):
    import kornia.color as kc  # fast RGB <-> HLS conversion

    img_tensor = img_tensor.unsqueeze(0)  # batch dim
    hls = kc.rgb_to_hls(img_tensor)

    noise = (torch.rand_like(hls) * 2 - 1) * noise_level
    hls_noisy = torch.clamp(hls + noise, 0.0, 1.0)

    rgb_noisy = kc.hls_to_rgb(hls_noisy)
    return rgb_noisy.squeeze(0)

# Process remaining images
for i, img_name in enumerate(files_to_process):
    img_path = os.path.join(source_dir, img_name)
    try:
        img = Image.open(img_path).convert("RGB")
        img_tensor = torch.tensor(np.array(img), dtype=torch.float32, device=device) / 255.0
        img_tensor = img_tensor.permute(2,0,1)  # C,H,W

        noisy_tensor = add_color_noise_hsl_gpu(img_tensor, noise_level=0.05)
        noisy_img = Image.fromarray((noisy_tensor.permute(1,2,0).cpu().numpy()*255).astype(np.uint8))

        save_path = os.path.join(target_dir, img_name)
        noisy_img.save(save_path)

        if (i+1) % 1000 == 0:
            print(f"{i+1} images processed")
    except Exception as e:
        print(f"Skipping {img_name}, error: {e}")


Images left to process: 23391
1000 images processed
2000 images processed
3000 images processed
4000 images processed
5000 images processed
6000 images processed
7000 images processed
8000 images processed
9000 images processed
10000 images processed
11000 images processed
12000 images processed
13000 images processed
14000 images processed
15000 images processed
16000 images processed
17000 images processed
18000 images processed
19000 images processed
20000 images processed
21000 images processed
22000 images processed
23000 images processed


In [48]:
folder = "natural_images/cut_corrupt/color_noise"  # your target folder
all_files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
num_images = len(all_files)

print(f"Number of images in {folder}: {num_images}")

Number of images in natural_images/cut_corrupt/color_noise: 35463


In [49]:
!du -sh natural_images/cut_corrupt/color_noise


237M	natural_images/cut_corrupt/color_noise


In [50]:
target_folder = "/content/drive/MyDrive/Samsung deep learning images/colorNoise"
os.makedirs(target_folder, exist_ok=True)


In [51]:
!cp -r "natural_images/cut_corrupt/color_noise" "/content/drive/MyDrive/Samsung deep learning images/colorNoise"
